# Analyse des Avis et Alertes ANSSI avec Enrichissement des CVE

## Étape 1 : Extraction des Flux RSS

In [ ]:
# faire "pip install feedparser" si besoin dans le terminal 
import feedparser

def recuperer_bulletins_rss(url):
    rss_feed = feedparser.parse(url)
    bulletins = []
    for entry in rss_feed.entries:
        bulletin = {
            "Titre du bulletin (ANSSI)": entry.title,
            "Description": entry.description,
            "Lien du bulletin (ANSSI)": entry.link,
            "Date de publication": entry.published
        }
        bulletins.append(bulletin)
    return bulletins

# URLs des flux RSS
avis_url = "https://www.cert.ssi.gouv.fr/avis/feed/"
alerte_url = "https://www.cert.ssi.gouv.fr/alerte/feed/"

avis_bulletins = recuperer_bulletins_rss(avis_url)
alerte_bulletins = recuperer_bulletins_rss(alerte_url)

bulletins = avis_bulletins + alerte_bulletins

# Afficher les bulletins pour vérification
for bulletin in bulletins:
    print(bulletin)

{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans Mattermost Server (13 mai 2025)', 'Description': 'De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.', 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +0000'}
{'Titre du bulletin (ANSSI)': 'Multiples vulnérabilités dans les produits SAP (13 mai 2025)', 'Description': "De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).", 'Lien du bulletin (ANSSI)': 'https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/', 'Date de publication': 'Tue, 13 May 2025 00:00:00 +00

## Étape 2 : Extraction des CVE

In [ ]:
import requests
import re

def extraire_cves_depuis_bulletin(lien_bulletin):
    # Construit l'URL JSON à partir du lien du bulletin
    if not lien_bulletin.endswith('/'):
        lien_bulletin += '/'
    url_json = lien_bulletin + "json/"
    response = requests.get(url_json)
    data = response.json()
    # Extraction des CVE référencés dans la clé "cves"
    ref_cves = [cve.get("name") for cve in data.get("cves", [])]
    #attention il s’agit d’une liste des dictionnaires avec name et url comme clés
    print( "CVE référencés ", ref_cves)
    # Extraction des CVE avec une regex
    cve_pattern = r"CVE-\d{4}-\d{4,7}"
    cve_list = list(set(re.findall(cve_pattern, str(data))))
    print("CVE trouvés :", cve_list)
    return ref_cves, cve_list

for bulletin in bulletins:
    ref_cves, cve_list = extraire_cves_depuis_bulletin(bulletin['Lien du bulletin (ANSSI)'])
    bulletin["CVEs"] = list(set(ref_cves + cve_list))
    print(f"Bulletin {bulletin['Titre du bulletin (ANSSI)']} : CVEs = {bulletin['CVEs']}")




CVE référencés  ['CVE-2025-4128', 'CVE-2025-4573']
CVE trouvés : ['CVE-2025-4573', 'CVE-2025-4128']
CVE référencés  ['CVE-2025-43003', 'CVE-2025-43007', 'CVE-2025-23191', 'CVE-2025-42999', 'CVE-2025-43009', 'CVE-2025-43011', 'CVE-2025-43006', 'CVE-2025-0060', 'CVE-2025-30012', 'CVE-2025-43000', 'CVE-2025-43004', 'CVE-2025-31324', 'CVE-2025-43005', 'CVE-2025-43008', 'CVE-2025-31329', 'CVE-2025-30009', 'CVE-2025-30011', 'CVE-2025-43002', 'CVE-2025-26662', 'CVE-2025-30010', 'CVE-2025-42997', 'CVE-2025-0061', 'CVE-2025-43010', 'CVE-2024-39592', 'CVE-2025-30018']
CVE trouvés : ['CVE-2025-0060', 'CVE-2025-30018', 'CVE-2025-43004', 'CVE-2025-43003', 'CVE-2025-0061', 'CVE-2025-30009', 'CVE-2025-43005', 'CVE-2025-30010', 'CVE-2025-43000', 'CVE-2025-31324', 'CVE-2025-30011', 'CVE-2025-23191', 'CVE-2025-42999', 'CVE-2025-43006', 'CVE-2025-43008', 'CVE-2025-43007', 'CVE-2025-43011', 'CVE-2025-43002', 'CVE-2025-43009', 'CVE-2025-43010', 'CVE-2025-26662', 'CVE-2024-39592', 'CVE-2025-42997', 'CVE-202

## Étape 3 : Enrichissement des CVE

### Exemple de connexion à l'API CVE :

In [3]:
# import requests
def extraire_infos_cve(cve_ids):
    resultats = []
    for cve_id in cve_ids:
        url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
        try:
            response = requests.get(url)
            data = response.json()
            # Description
            description = data["containers"]["cna"]["descriptions"][0]["value"]
            # Score CVSS (gérer les différentes versions et exceptions)
            cvss_score = None
            metrics = data["containers"]["cna"].get("metrics", [])
            if metrics:
                metric = metrics[0]
                for key in ["cvssV3_1", "cvssV3_0", "cvssV2_0"]:
                    if key in metric:
                        cvss_score = metric[key].get("baseScore")
                        break
            # CWE
            cwe = "Non disponible"
            cwe_desc = "Non disponible"
            problemtype = data["containers"]["cna"].get("problemTypes", [])
            if problemtype and "descriptions" in problemtype[0]:
                cwe = problemtype[0]["descriptions"][0].get("cweId", "Non disponible")
                cwe_desc = problemtype[0]["descriptions"][0].get("description", "Non disponible")
            # Produits affectés
            produits = []
            for product in data["containers"]["cna"].get("affected", []):
                vendor = product.get("vendor", "Non disponible")
                product_name = product.get("product", "Non disponible")
                versions = [v["version"] for v in product.get("versions", []) if v.get("status") == "affected"]
                produits.append({
                    "Éditeur": vendor,
                    "Produit": product_name,
                    "Versions": versions
                })
            # Stockage des résultats
            resultats.append({
                "CVE": cve_id,
                "Description": description,
                "Score CVSS": cvss_score,
                "Type CWE": cwe,
                "CWE Description": cwe_desc,
                "Produits affectés": produits
            })
        except Exception as e:
            print(f"Erreur pour {cve_id} : {e}")
    return resultats

tous_les_cves = list(set(ref_cves + cve_list))

infos_cve = extraire_infos_cve(tous_les_cves)
for info in infos_cve:
    print(f"CVE : {info['CVE']}")
    print(f"Description : {info['Description']}")
    print(f"Score CVSS : {info['Score CVSS']}")
    print(f"Type CWE : {info['Type CWE']}")
    print(f"CWE Description : {info['CWE Description']}")
    for produit in info["Produits affectés"]:
        print(f"Éditeur : {produit['Éditeur']}, Produit : {produit['Produit']}, Versions : {', '.join(produit['Versions'])}")
    print("-" * 40)

Erreur pour CVE-2024-47177 : 'descriptions'
CVE : CVE-2020-9548
Description : FasterXML jackson-databind 2.x before 2.9.10.4 mishandles the interaction between serialization gadgets and typing, related to br.com.anteros.dbcp.AnterosDBCPConfig (aka anteros-core).
Score CVSS : None
Type CWE : Non disponible
CWE Description : n/a
Éditeur : n/a, Produit : n/a, Versions : n/a
----------------------------------------
CVE : CVE-2019-16335
Description : A Polymorphic Typing issue was discovered in FasterXML jackson-databind before 2.9.10. It is related to com.zaxxer.hikari.HikariDataSource. This is a different vulnerability than CVE-2019-14540.
Score CVSS : None
Type CWE : Non disponible
CWE Description : n/a
Éditeur : n/a, Produit : n/a, Versions : n/a
----------------------------------------
CVE : CVE-2019-17267
Description : A Polymorphic Typing issue was discovered in FasterXML jackson-databind before 2.9.10. It is related to net.sf.ehcache.hibernate.EhcacheJtaTransactionManagerLookup.
Sco

### Exemple de connexion à l’API EPSS:

In [4]:
#import requests
def extraire_epss_pour_cves(cve_ids):
    epss_scores = {}
    # URL de l'API EPSS pour récupérer la probabilité d'exploitation
    for cve_id in cve_ids:
        url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
        try:
            # Requête GET pour récupérer les données JSON
            response = requests.get(url)
            data = response.json()
            # Extraire le score EPSS
            epss_data = data.get("data", [])
            if epss_data:
                epss_score = epss_data[0].get("epss")
                epss_scores[cve_id] = epss_score
                print(f"CVE : {cve_id}")
                print(f"Score EPSS : {epss_score}")
            else:
                epss_scores[cve_id] = None
                print(f"Aucun score EPSS trouvé pour {cve_id}")
        except Exception as e:
            epss_scores[cve_id] = None
            print(f"Erreur pour {cve_id} : {e}")
    return epss_scores

epss_scores = extraire_epss_pour_cves(tous_les_cves)


CVE : CVE-2020-9548
Score EPSS : 0.139450000
CVE : CVE-2019-16335
Score EPSS : 0.007400000
CVE : CVE-2019-17267
Score EPSS : 0.013570000
CVE : CVE-2020-9492
Score EPSS : 0.001150000
CVE : CVE-2025-27363
Score EPSS : 0.757480000
CVE : CVE-2017-7525
Score EPSS : 0.773360000
CVE : CVE-2025-0917
Score EPSS : 0.000240000
CVE : CVE-2025-27220
Score EPSS : 0.001260000
CVE : CVE-2022-26612
Score EPSS : 0.001500000
CVE : CVE-2024-45296
Score EPSS : 0.001310000
CVE : CVE-2025-22870
Score EPSS : 0.000120000
CVE : CVE-2025-24928
Score EPSS : 0.000070000
CVE : CVE-2023-45178
Score EPSS : 0.000350000
CVE : CVE-2018-19361
Score EPSS : 0.040630000
CVE : CVE-2024-1975
Score EPSS : 0.003870000
CVE : CVE-2022-25168
Score EPSS : 0.028270000
CVE : CVE-2018-14718
Score EPSS : 0.147470000
CVE : CVE-2024-53382
Score EPSS : 0.000410000
CVE : CVE-2019-14540
Score EPSS : 0.070820000
CVE : CVE-2019-14893
Score EPSS : 0.006980000
CVE : CVE-2024-12905
Score EPSS : 0.004440000
CVE : CVE-2022-3510
Score EPSS : 0.0005

## Étape 4 : Consolidation des Données

In [5]:
import pandas as pd

# Supposons que tu as déjà exécuté les fonctions suivantes et stocké les résultats dans ces variables :
# bulletins, ref_cves, cve_list, infos_cve, epss_scores

# bulletins = [...]      # Liste de bulletins (dictionnaires)
# ref_cves = [...]       # Liste des CVE référencés (extraites)
# cve_list = [...]       # Liste des CVE trouvés par regex
# infos_cve = [...]      # Liste des infos enrichies pour chaque CVE
# epss_scores = {...}    # Dictionnaire {cve: score}


infos_cve_dict = {info["CVE"]: info for info in infos_cve}

donnees = []
for bulletin in bulletins:
    cves_bulletin = bulletin.get("CVEs", [])
    if not cves_bulletin:
        # ligne par défaut
        donnees.append({
            "ID du bulletin (ANSSI)": bulletin.get("ID du bulletin (ANSSI)", ""),
            "Titre du bulletin (ANSSI)": bulletin.get("Titre du bulletin (ANSSI)", ""),
            "Type de bulletin": bulletin.get("Type de bulletin", ""),
            "Date de publication": bulletin.get("Date de publication", ""),
            "Identifiant CVE": None,
            "Score CVSS": None,
            "Base Severity": None,
            "Type CWE": None,
            "Score EPSS": None,
            "Lien du bulletin (ANSSI)": bulletin.get("Lien du bulletin (ANSSI)", ""),
            "Description": bulletin.get("Description", ""),
            "Éditeur/Vendor": None,
            "Produit": None,
            "Versions affectées": None
        })
    else:
        for cve in cves_bulletin:
            # enrichissement
            info = infos_cve_dict.get(cve, {})
            produits = info.get("Produits affectés", [{}])
            produit = produits[0] if produits else {}
            donnees.append({
                "ID du bulletin (ANSSI)": bulletin.get("ID du bulletin (ANSSI)", ""),
                "Titre du bulletin (ANSSI)": bulletin.get("Titre du bulletin (ANSSI)", ""),
                "Type de bulletin": bulletin.get("Type de bulletin", ""),
                "Date de publication": bulletin.get("Date de publication", ""),
                "Identifiant CVE": cve,
                "Score CVSS": info.get("Score CVSS"),
                "Base Severity": None,
                "Type CWE": info.get("Type CWE"),
                "Score EPSS": epss_scores.get(cve),
                "Lien du bulletin (ANSSI)": bulletin.get("Lien du bulletin (ANSSI)", ""),
                "Description": info.get("Description", bulletin.get("Description", "")),
                "Éditeur/Vendor": produit.get("Éditeur"),
                "Produit": produit.get("Produit"),
                "Versions affectées": ", ".join(produit.get("Versions", [])) if produit.get("Versions") else None
            })

df = pd.DataFrame(donnees)
df


,ID du bulletin (ANSSI),Titre du bulletin (ANSSI),Type de bulletin,Date de publication,Identifiant CVE,Score CVSS,Base Severity,Type CWE,Score EPSS,Lien du bulletin (ANSSI),Description,Éditeur/Vendor,Produit,Versions affectées
0,,Multiples vulnérabilités dans Mattermost Serve...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-4573,NaN,None,None,None,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,De multiples vulnérabilités ont été découverte...,None,None,None
1,,Multiples vulnérabilités dans Mattermost Serve...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-4128,NaN,None,None,None,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,De multiples vulnérabilités ont été découverte...,None,None,None
2,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-0060,NaN,None,None,None,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,De multiples vulnérabilités ont été découverte...,None,None,None
3,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-30018,NaN,None,None,None,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,De multiples vulnérabilités ont été découverte...,None,None,None
4,,Multiples vulnérabilités dans les produits SAP...,,"Tue, 13 May 2025 00:00:00 +0000",CVE-2025-43004,NaN,None,None,None,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,De multiples vulnérabilités ont été découverte...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,,Multiples vulnérabilités dans les produits IBM...,,"Fri, 13 Jun 2025 00:00:00 +0000",CVE-2024-37529,6.5,None,CWE-789,0.001320000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,"IBM Db2 for Linux, UNIX and Windows (includes ...",IBM,"Db2 for Linux, UNIX and Windows","11.1, 11.5"
1206,,Multiples vulnérabilités dans les produits IBM...,,"Fri, 13 Jun 2025 00:00:00 +0000",CVE-2025-22868,NaN,None,Non disponible,0.000630000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,An attacker can pass a malicious malformed tok...,golang.org/x/oauth2,golang.org/x/oauth2/jws,0
1207,,Multiples vulnérabilités dans les produits IBM...,,"Fri, 13 Jun 2025 00:00:00 +0000",CVE-2018-14721,NaN,None,Non disponible,0.095990000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,FasterXML jackson-databind 2.x before 2.9.7 mi...,n/a,n/a,n/a
1208,,Multiples vulnérabilités dans les produits IBM...,,"Fri, 13 Jun 2025 00:00:00 +0000",CVE-2024-25062,NaN,None,Non disponible,0.001550000,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,An issue was discovered in libxml2 before 2.11...,n/a,n/a,n/a
